In [1]:
import pandas as pd
import numpy as np
import config
import datetime
from sqlalchemy import text
import plotly
import plotly.express as px
import ipywidgets
from urllib.request import urlopen
import json
import plotly.graph_objects as go
import geopandas as gpd
import warnings
import pyproj
import plotly.io as pio
import pickle
print(config.version)
print(config.connection_string)
print(plotly.__version__, ipywidgets.__version__, pd.__version__, np.__version__)
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore', category=UserWarning) 
pd.options.mode.copy_on_write = True


3.2.0.Dev
postgresql://postgres:sa@localhost/icip
6.3.1 8.1.7 2.0.0 1.26.4


In [2]:
gdf_france = gpd.read_file("communes-100m-2024.geojson")
gdf_france=gdf_france[gdf_france["departement"].str.len()==2]
geojson=gdf_france.__geo_interface__
gdf_france

,code,nom,departement,region,commune,plm,epci,geometry
0,01001,L'Abergement-Clémenciat,01,84,None,NaN,200069193,"POLYGON ((4.9584 46.1533, 4.9516 46.1474, 4.95..."
1,01002,L'Abergement-de-Varey,01,84,None,NaN,240100883,"POLYGON ((5.4302 45.9828, 5.4245 45.9865, 5.42..."
2,01004,Ambérieu-en-Bugey,01,84,None,NaN,240100883,"POLYGON ((5.4088 45.9421, 5.4008 45.9427, 5.39..."
3,01005,Ambérieux-en-Dombes,01,84,None,NaN,200042497,"POLYGON ((4.943 45.9796, 4.9373 45.9789, 4.936..."
4,01006,Ambléon,01,84,None,NaN,200040350,"POLYGON ((5.5708 45.7534, 5.574 45.7539, 5.579..."
...,...,...,...,...,...,...,...,...
34846,95676,Villers-en-Arthies,95,11,None,NaN,249500513,"POLYGON ((1.741 49.0992, 1.7427 49.0943, 1.753..."
34847,95678,Villiers-Adam,95,11,None,NaN,249500455,"POLYGON ((2.2739 49.0766, 2.27 49.0751, 2.2699..."
34848,95680,Villiers-le-Bel,95,11,None,NaN,200055655,"POLYGON ((2.41 49.0241, 2.4108 49.0241, 2.4122..."
34849,95682,Villiers-le-Sec,95,11,None,NaN,200073013,"POLYGON ((2.3962 49.0867, 2.4014 49.083, 2.405..."


In [3]:
years = list(range(13,25))
bor_dico = {"urgence_gen": "Urgences générales", "urgence_ped": "Urgences prédiatriques", "psy": "Etablissements psychiatriques", "pharma": "Pharmacies", "ehpad": "EHPAD"}

Documentation

code_commune: il y a plusieurs possibilités :
* le code de la commune sur 5 chiffres, par exemple "06088" pour Nice
* certains anciens code commune sont acceptés comme 75101 pour Paris 1er arrondissement
* le département sur 2 chiffres, par exemple "06"
* le code IRIS sur 9 chiffres, par exemple "382050000"
* une liste de code IRIS entre crochet, par exemple \["385480000", "382050000"]
* une liste de code commune entre crochet, par exemple \["38548", "38205"]
* le code IRIS avec un joker sql de type "3820%"
* pour la corse utiliser les préfixes 2A ou 2B et non 20, par exemple "2A004"

bor: voir tableau ci dessus

time: 30 ou 45, il est possible d'en faire d'autre mais il faut lancer les calculs

time_type: "HC" ou "HP"

aexp: -0.12 ou -0.1 ou -0.08 ou -0.06 ou -0.04<br>
Pour time==30 aexp: -0.12 ou -0.1 ou -0.08<br>
Pour time==45 aexp: -0.08 ou -0.06<br>
Pour time==60 aexp: -0.06 ou -0.04

norm_colorbar = True si normalisé sur France entière False sinon

meanw_colorbar = True si normalisé sur moyenne pondérée, False si normalisé sur Médiane

Pour ne plus voir les labels sur les IRIS mettre limit_show=0

Puis appuyer sur le bouton Restart (bouton avec double flèche à droite FFW) ou dans le menu Run / Run selected cell and all below

In [61]:
code_commune="75101" #"75101" #"75056" #"69123" #"38185"  #"06088" "38205" "38548" "2A004" "382050000" ["385480000", "382050000"] ["38548", "38205"] "45" "14581" bug
bor = "ehpad"
time=60
time_type="HC" # HC or HP
norm_colorbar=False
meanw_colorbar=True
code_commune,bor,time,time_type,norm_colorbar,meanw_colorbar

('75101', 'ehpad', 60, 'HC', False, True)

In [62]:
if type(code_commune) is list:
    if len(code_commune[0])==5:
        type_code="CLIST"
    else:
        raise ValueError(f"Bad {code_commune}")    
elif "%" in code_commune and len(code_commune) > 2:
    type_code = "LIKE"
elif len(code_commune) == 2:
    type_code = "LIKE"
    code_commune = code_commune + "%"
elif len(code_commune) == 5:
    type_code = "COM"
else:
    raise ValueError(f"Bad {code_commune}")
print(f"Type code: {type_code}")

Type code: COM


In [63]:
sql = f"""
select i.*, c.nom commune_nom, c.code code_commune from iris.commune c
join iris.iris i on i.commune_id=c.id
where c.code='{code_commune}'
order by i.id
"""
commune_nom = None
if type_code=="COM":
    iris_df = pd.read_sql(sql, config.connection_string)
    iriss = iris_df["code"].values
    code_communes = iris_df["code_commune"].unique()
    commune_nom = iris_df["commune_nom"].iloc[0] if len(iris_df) > 0 else None
elif type_code in ["LIKE"]:
    iriss = [code_commune]
elif type_code == "CLIST":
    sql = f"""
select i.*, c.nom commune_nom, c.code code_commune from iris.commune c
join iris.iris i on i.commune_id=c.id
where c.code in {tuple(code_commune)}
"""
    print(sql)
    iris_df = pd.read_sql(sql, config.connection_string)
    iriss = iris_df["code"].values
    code_communes = iris_df["code_commune"].unique()
    commune_nom = iris_df["commune_nom"].iloc[0] if len(iris_df) > 0 else None
print(f"Found {len(iriss)} iris and {len(code_communes)} communes for {commune_nom}")

if type_code == "LIKE":
    gdf = gdf_france[gdf_france["code"].str.startswith(code_commune[:-1])]
elif len(iriss)>0:
    gdf = gdf_france[gdf_france["code"].isin(code_communes)]
else:
    gdf = gdf_france[gdf_france["code_insee"]==code_commune]
    type_code="INSEE"
if commune_nom is None:
    commune_nom = code_commune
    # commune_nom=gdf["nom_commune"].iloc[0] if len(gdf) > 0 else None
    # print(f"Found {len(gdf)} iris in {commune_nom}")
geojson=gdf.__geo_interface__
print(f"Found {len(gdf)} geo iris for {commune_nom[:-1]}, type_code: {type_code}")
# gdf_france[gdf_france["nom_commune"].str.startswith("Lyon")].head(100)
gdf

Found 0 iris and 0 communes for None


KeyError: 'code_insee'

In [ ]:
def get_study_by_year(year):
    sql = f"""
    select * from sae.dist_study s
    where s.bor='{bor}'
    and s.year=20{year}
    and s.time={time}
    and s.time_type='{time_type}'
    order by date desc
    limit 1
    """
    return pd.read_sql(sql, config.connection_string)

def get_studies_by_years(years: list[int]):
    df = None
    for year in years:
        if df is None:
            df = get_study_by_year(year)
        else:
            df = pd.concat([df, get_study_by_year(year)], ignore_index=True)
    return df
        
        
    
studies_df = get_studies_by_years(years)
studies_df

In [ ]:
keys = studies_df["key"].values
print(f"Studies keys: {keys}")
if type_code == "COM":
    sql=f"""
    select * from sae.dist a
    where a.study_key in {tuple(keys)}
    and code_commune='{code_commune}'
    """
elif type_code == "LIKE":
    sql=f"""
    select * from sae.dist a
    where a.study_key in {tuple(keys)}
    and a.iris_string like '{code_commune}'
    """
elif type_code == "CLIST":
    sql=f"""
    select * from sae.dist a
    where a.study_key in {tuple(keys)}
    and a.code_commune in {tuple(code_commune)}
    """
print(sql)
apl = pd.read_sql(text(sql), config.connection_string)
if type_code == "INSEE" or len(apl) == 0:
    sql=f"""
    select * from sae.dist a
    where a.study_key in {tuple(keys)}
    """
    apl = pd.read_sql(sql, config.connection_string)
apl

In [ ]:
def get_urgence_sae(iris_list, urg="GEN"):
    sql = f"""
select d.an as year, d.fi, d.passu, p.perso is not null has_pdata, p.etpsal, p.efflib, p.etp, e.id etab_id, e.rs, an.dept_id, an.id adresse_norm_id, an.lon, an.lat, i.id iris 
from sae.urgence_detail d
left join sae.urgence_p p on p.fi=d.fi and p.an=d.an and p.perso='M9999'
join etablissement e on e.nofinesset=d.fi
join adresse_raw ar on e.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join iris.iris i on i.code=an.iris
where d.urg='{urg}'
and an.iris in {tuple(iris_list)}
order by d.an, d.fi
"""
    print(f"Quering SAE urgence for {urg}")
    # print(sql)
    return pd.read_sql(sql, config.connection_string)

def get_psy_sae(iris_list):
    sql = f"""
select p.an as year, p.fi, p.cap_htp passu, true has_pdata, p.etpsal_pkt etpsal, p.efflib_pkt efflib, p.etp_pkt etp, e.id etab_id, e.rs, an.dept_id, an.id adresse_norm_id, an.lon, an.lat, i.id iris 
from sae.psy p
join etablissement e on e.nofinesset=p.fi
join adresse_raw ar on e.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join iris.iris i on i.code=an.iris
where p.dis='TOT'
and an.iris in {tuple(iris_list)}
order by p.an, p.fi
"""
    print(f"Quering SAE psy")
    # print(sql)
    return pd.read_sql(sql, config.connection_string)

def get_pharma(iris_list):
    sql = f"""
select ds.annee+2000 as year, e.nofinesset fi, null as passu, false as has_pdata, null as etpsal, null as efflib, null as etp, e.id etab_id, e.rs, an.id an_id, an.dept_id, an.id adresse_norm_id, an.lon, an.lat, i.id iris
from etablissement e
join etablissement_date_source eds on eds.etablissement_id=e.id
join adresse_raw ar on ar.id=e.adresse_raw_id
join adresse_norm an on an.id=ar.adresse_norm_id
join iris.iris i on i.code=an.iris
join date_source ds on eds.date_source_id=ds.id
where e.categetab=620
and an.iris in {tuple(iris_list)}
and ds.annee>=13
order by ds.annee, e.nofinesset
    """
    print(f"Quering etablissement pharma")
    # print(sql)
    return pd.read_sql(sql, config.connection_string)

def get_ehpad(commune_list):
    sql = f"""
select ds.annee+2000 as year, e.nofinesset fi, null as passu, false as has_pdata, null as etpsal, null as efflib, null as etp, e.id etab_id, e.rs, an.id an_id, an.dept_id, an.id adresse_norm_id, an.lon, an.lat, i.id iris, c.id commune_id, c.code code_commune
from etablissement e
join etablissement_date_source eds on eds.etablissement_id=e.id
join adresse_raw ar on ar.id=e.adresse_raw_id
join adresse_norm an on an.id=ar.adresse_norm_id
join iris.iris i on i.code=an.iris
join iris.commune c on c.id=i.commune_id
join date_source ds on eds.date_source_id=ds.id
where e.categretab=4401
and c.code in {tuple(commune_list) if len(commune_list)!=1 else "('"+ commune_list[0]+"')"}
and ds.annee>=13
order by ds.annee, e.nofinesset
    """
    print(f"Quering etablissement ehpad")
    # print(sql)
    return pd.read_sql(sql, config.connection_string)


def get_sae_by_bor(bor, commune_list):
    if bor == "urgence_gen":
        return get_urgence_sae(commune_list, "GEN")
    elif bor == "urgence_ped":
        return get_urgence_sae(commune_list, "PED")
    elif bor == "psy":
        return get_psy_sae(commune_list)
    elif bor == "pharma":
        return get_pharma(commune_list)
    elif bor == "ehpad":
        return get_ehpad(commune_list)
        

commune_list = list(gdf["code"].unique())
etab_df = get_sae_by_bor(bor, commune_list)
etab_df

In [ ]:
etab_df2 = etab_df.drop_duplicates(subset=['year', "lon", "lat"])
etab_df2["tension"] = etab_df2["passu"]/etab_df2["etp"]
etab_df2["has_pdata"] = etab_df2["tension"].notna() 
etab_df2

In [ ]:
# etab = apl[apl["km"]==0].drop_duplicates(subset=['year', "lon", "lat"])
# etab = etab[["year","fi","rs","passu","has_pdata","etpsal","efflib","etp","tension","lon","lat"]]
# etab = etab.dropna(subset=["lon"])
# etab
etab = etab_df2[["year","fi","rs","passu","has_pdata","etpsal","efflib","etp","tension","lon","lat"]]

In [ ]:
gdf

In [ ]:
score_col = "time_hc"
apl=apl.sort_values(by=["year","iris"])
apl["pop_sum"] = apl['pop'].groupby([apl["code_commune"], apl["year"]]).transform('sum')
apl[f"{score_col}_meanw"] = (apl[score_col].fillna(90) * apl['pop']).groupby([apl["code_commune"], apl["year"]]).transform('sum') / apl["pop_sum"]
apl[f"{score_col}_meanw"] = apl[f"{score_col}_meanw"].mask(apl[f"{score_col}_meanw"] > 60, np.nan)
# apl[apl["code_commune"]=="06088"]
apl

In [ ]:
apl2 = apl.drop_duplicates(subset=['year', "code_commune"])
apl2

In [ ]:
gdf["code_commune"]=gdf["code"]
gdf_merged = gdf.merge(apl2, on="code_commune", how="left", suffixes=('', '_dest')).copy()
print(gdf_merged.columns)
apl13 = gdf_merged.loc[gdf_merged['year']==2013, ['iris', 'time_hc']].set_index('iris')['time_hc']
gdf_merged["year13"]=gdf_merged['iris'].map(apl13)
gdf_merged["diff13"]=gdf_merged["time_hc"]-gdf_merged["year13"]
gdf_merged["delta13"]=gdf_merged["diff13"]/(gdf_merged["year13"]+1e-5)
gdf_merged

In [ ]:
q10 = studies_df[studies_df["year"]==2013]["q10"].iloc[0]
q25 = studies_df[studies_df["year"]==2013]["q25"].iloc[0]
q50 = studies_df[studies_df["year"]==2013]["q50"].iloc[0]
q75 = studies_df[studies_df["year"]==2013]["q75"].iloc[0]
q90 = studies_df[studies_df["year"]==2013]["q90"].iloc[0]
qmax = 60 if bor not in ["pharma", "ehpad"] else 30
if bor=="pharma":
    q10 += 1
    q25 += 2
elif bor=="ehpad":
    q10 += 1
q10,q25,q50,q75,q90,qmax

In [ ]:
if bor in ["pharma", "ehpad"]:
    etab = etab[etab['year'] != 2018]
    rows_2017 = etab[etab['year'] == 2017].copy()
    rows_2018 = rows_2017.assign(year=2018)
    etab = pd.concat([etab, rows_2018], ignore_index=True)
    gdf_merged = gdf_merged[gdf_merged['year'] != 2018]
    rows_2017 = gdf_merged[gdf_merged['year'] == 2017].copy()
    rows_2018 = rows_2017.assign(year=2018)
    gdf_merged = pd.concat([gdf_merged, rows_2018], ignore_index=True)
    gdf_merged[gdf_merged["year"]==2018]

In [ ]:
etab[etab["year"]==2022]

In [ ]:
show_label=False
fig = go.Figure()
l=[]
l2=[]
center_lat = gdf.geometry.centroid.y.mean()
center_lon = gdf.geometry.centroid.x.mean()
passage_dico = {"urgence_gen": "passage", "urgence_ped": "passage", "psy": "lit", "pharma": "passage", "ehpad": "lit"}
size_dico = {"urgence_gen": 8/30000, "urgence_ped": 8/30000, "psy": 6/100, "pharma": 1, "ehpad": 1}
for year in years:
    gdf_year=gdf_merged[gdf_merged["year"]==year+2000]
    l.append(
        go.Choropleth(
            visible=False,
            geojson=geojson,
            featureidkey="properties.code",
            locations=gdf_year["code"],
            z=gdf_year["time_hc_meanw"].fillna(60),
            zmin=0,
            zmax=qmax,
            text=gdf_year["time_hc_meanw"].fillna("NA"),
#             text=gdf_year.apply(
#                 lambda row: f"""
# Commune: {row["nom_commune"]}<br>
# Iris: {row["code_iris"]}<br>
# Nom Iris: {row['nom_iris']}<br>
# Etablissement le plus proche: {"Aucun" if row["rs"] is None else row["rs"].capitalize()} <br>
# Temps d'accès: {">60" if np.isnan(row["time_hc"]) else int(row["time_hc"])} min<br>
# Temps d'accès heure pleine: {">60" if np.isnan(row["time_hp"]) else int(row["time_hp"])} min<br>
# Distance: {"NA" if np.isnan(row["km"]) else int(row["km"])} km<br>
# Variation / 2013: {"NA" if np.isnan(row["delta13"]) else int(row["delta13"]*100)}%<br>
# Population: {row['pop']:.0f}<br>""",
#                 axis=1
#             ),
            hoverinfo="text",
            showscale=True,
            colorbar={"title": f"Durée (min)"},
            colorscale=[[0.0, "rgb(127,0,0)"],
                        [q10/qmax if norm_colorbar else 0.1, "rgb(187,64,55)"],
                        [q25/qmax if norm_colorbar else 0.25, "rgb(209,127,79)"],
                        [q50/qmax if norm_colorbar else 0.5, "rgb(255,255,127)"],
                        [q75/qmax if norm_colorbar else 0.75, "rgb(159,159,127)"],
                        [q90/qmax if norm_colorbar else 0.9, "rgb(112,112,127)"],
                        [1.0, "rgb(64,64,127)"]
                       ]
        ))
    if show_label:
        etab_year=etab[etab["year"]==year+2000]
#         if bor in ["urgence_gen", "urgence_ped"]:
#             colors = etab_year.apply(lambda row: 
# f"rgb({127 if not(row["has_pdata"]) or row["tension"]==np.inf else np.clip(int((row['tension']-3000)*0.1+127.5),0,255)},{255 - (127 if not(row["has_pdata"]) or row["tension"]==np.inf else np.clip(int((row['tension']-3000)*0.1+127.5),0,255))},0)"
#                 , axis=1)
#         else:
#             colors = etab_year.apply(lambda row: 
# f"rgb({127 if not(row["has_pdata"]) or row["tension"]==np.inf else np.clip(int((row['tension']-11)*10+127.5),0,255)},{255 - (127 if not(row["has_pdata"]) or row["tension"]==np.inf else np.clip(int((row['tension']-3000)*0.1+127.5),0,255))},0)"                                              
#                 , axis=1)
#         l2.append(
#             go.Scattergeo(
#                 lon=etab_year["lon"],
#                 lat=etab_year["lat"],
#                 text=etab_year.apply(lambda row: f"""
# {row["rs"]}<br>
# N° Finess: {row["fi"]}<br>
# Nb {passage_dico[bor]} 20{year}: {"NA" if row["passu"] is None or math.isnan(row["passu"]) else int(row["passu"])}<br>
# ETP: {"NA" if not(row["has_pdata"]) else np.round(row["etp"],1)}<br>
# Dont salarié: {"NA" if not(row["has_pdata"]) else np.round(row["etpsal"],1)}<br>
# Dont libéraux: {"NA" if not(row["has_pdata"]) else np.round(row["efflib"],1)}<br>
# Tension: {"NA" if not(row["has_pdata"]) or row["tension"]==np.inf else int(row["tension"])} {passage_dico[bor]}/ETP<br>
# GPS: {row["lat"]},{row["lon"]}<br>
# Color: rgb({127 if not(row["has_pdata"]) or row["tension"]==np.inf else np.clip(int((row['tension']-3000)*0.1+127.5),0,255)},{255 - (127 if not(row["has_pdata"]) or row["tension"]==np.inf else np.clip(int((row['tension']-3000)*0.1+127.5),0,255))},0)
# """, axis=1),
#                 hoverinfo="text",
#                 showlegend=False,
#                 visible=False,
#                 marker=dict(
#                         color=colors,
#                         size=(etab_year["passu"].fillna(0))*size_dico[bor]+2,
#                         opacity=1,
#                         line=dict(
#                             color='#99ff99',
#                             width=2
#                         ),
#                     ),
#             ))
fig = go.Figure(data=l+l2)
i_visible = 0
fig.data[i_visible].visible = True
if show_label:
    fig.data[len(l)+i_visible].visible = True
steps = []
i=0
for year in years:
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
             ],
        label=f"20{year}",
    )
    step["args"][0]["visible"][i] = True
    if show_label:
        step["args"][0]["visible"][i+len(l)] = True
    steps.append(step)
    i+=1

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Durée de trajet ", "visible":True},
    steps=steps,
    pad=dict(l=0, r=0, t=0, b=10),
    len=0.6,
    x=0.2,
    y=0.05,
)]

fig.update_layout(
    sliders=sliders,
    geo=dict(
        projection_type="mercator",
        center={"lat": center_lat, "lon": center_lon},
        fitbounds="locations",
        showland=False,
        showcountries=False,
        showocean=False,
        showlakes=False,
        showrivers=False,
        visible=False,
    ),
    title=f"Durée de trajet par Iris - {bor_dico[bor]} - {"Ville de" if len(code_commune) != 3 else "Département"} {commune_nom if len(code_commune) != 3 else code_commune[:-1]}",
    height=600,
    # width=2000,  
    autosize=False,
    margin=dict(l=10, r=0, t=50, b=10),
)

fig.show()

In [ ]:
gdf_merged["time_hc"].describe().apply("{0:.0f}".format)

In [ ]:
fig.update_layout(
    height=None,
    width=None, 
    autosize=True,
    # margin=dict(l=10, r=0, t=50, b=50)
)
pio.write_html(fig, file=f"out/sae_dist_{bor}_{code_commune.replace("%","")}.html", auto_open=True, include_plotlyjs="cdn", full_html=True, config={"responsive": True})